<a href="https://colab.research.google.com/github/sahil3Vedi/Enron-Email-Classifier/blob/master/ActiveTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
from sklearn.svm import LinearSVC
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
def preprocess(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)

  return x, y

def create_dataset(xs, ys, n_classes=10):
  ys = tf.one_hot(ys, depth=n_classes)
  return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(500)

In [5]:
#TRAINING
ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ANNO_LOC = 'drive/My Drive/Enron Spam/annotations.npy'
ANNOIND_LOC = 'drive/My Drive/Enron Spam/annotation_indices.npy'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'

features = np.load(FEATURES_LOC)
ANNO = np.load(ANNOIND_LOC)
X_train = []
for each_index in ANNO:
  X_train.append(features[each_index])
Y_load = np.load(ANNO_LOC)

Y_train = [each for each in Y_load]

model1 = LinearSVC()
model1.fit(X_train, Y_train)


y_binary = to_categorical(Y_train)

XT = np.asarray(X_train)
print(np.shape(XT))
print(np.shape(y_binary))


model2 = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(1000,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add an output layer with 10 output units:
layers.Dense(2, activation='softmax')])

model2.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model2.fit(XT, y_binary, epochs=10, batch_size=32)


(50, 1000)
(50, 2)
Epoch 1/10
50/50 [==============================] - 0s 7ms/sample - loss: 0.6990 - accuracy: 0.4000
Epoch 2/10
50/50 [==============================] - 0s 180us/sample - loss: 0.5568 - accuracy: 0.8000
Epoch 3/10
50/50 [==============================] - 0s 276us/sample - loss: 0.4571 - accuracy: 0.9400
Epoch 4/10
50/50 [==============================] - 0s 240us/sample - loss: 0.3829 - accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - 0s 227us/sample - loss: 0.3395 - accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 0s 206us/sample - loss: 0.3206 - accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 0s 185us/sample - loss: 0.3158 - accuracy: 1.0000
Epoch 8/10
50/50 [==============================] - 0s 189us/sample - loss: 0.3140 - accuracy: 1.0000
Epoch 9/10
50/50 [==============================] - 0s 208us/sample - loss: 0.3135 - accuracy: 1.0000
Epoch 10/10
50/50 [==============================] - 0s 187us/sam

In [0]:
#TESTING
def fetchLabel(mailIndex):
  twoD_array = []
  feature_vector = features[mailIndex]
  twoD_array.append(feature_vector)
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    #print("Message Contents: ")
    #print(data)
    #print("Classifier Label: ")
    result1 = model1.predict(twoD_array)[0]
    DD = np.asarray(twoD_array)
    #print(np.shape(DD))
    result2 = model2.predict(DD)
    fetch_results = [result1,result2]
    return fetch_results

In [8]:
results1 = []
results2 = []
for x in range(10):
  labels = fetchLabel(x)
  results1.append(labels[0])
  results2.append(labels[1])
print(results1)
print(results2)

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[array([[1.000000e+00, 2.984492e-15]], dtype=float32), array([[9.9998498e-01, 1.4963661e-05]], dtype=float32), array([[1.0000000e+00, 1.8749707e-12]], dtype=float32), array([[0.8725409 , 0.12745917]], dtype=float32), array([[0.99870014, 0.00129979]], dtype=float32), array([[4.239726e-10, 1.000000e+00]], dtype=float32), array([[0.7820189 , 0.21798109]], dtype=float32), array([[9.999999e-01, 9.409231e-08]], dtype=float32), array([[9.9997187e-01, 2.8140907e-05]], dtype=float32), array([[9.9999952e-01, 5.3027327e-07]], dtype=float32)]
